# 12.5.2 Continuous monitoring and finite discrete state space

This notebook is dedicated to an extension of the *Case 4* where the "item" it taken out of operation during the maintenance. This case is depicted figure 12.11.

**NOTE:** In the following, states $0^R, 1^R$ and $2^R$ are respectively labeled states 4, 5 and 6. Transition labels have been changed accordingly. Resulting notations are provided through the scheme below.

![](./../../images/Schema_12_05_02.png)


In [1]:
import numpy as np
import pandas as pd
from scipy.interpolate import interp1d

%matplotlib notebook
import matplotlib.pyplot as plt

In [2]:
def RandExp(rate):
    '''Draw random samples from an exponential distribution.
    If rate is null, output is set to infinity.'''
    if rate==0:
        return np.inf
    else:
        return np.random.exponential(scale=1/rate)

In [3]:
def UpdateVar(horizonTime, currentTime, currentState, dT, nextState, 
              timeState, transMatrix):
    '''Update system state and time spent in the different states and achieved transitions.'''
    currentTime = currentTime+dT
    flagSim = True
    if currentTime>=horizonTime:
        flagSim = False
        dT = dT-(currentTime-horizonTime)
        currentTime = currentTime+dT
    timeState[currentState] = timeState[currentState]+dT
    transMatrix[currentState,nextState] = transMatrix[currentState,nextState]+1
    currentState = nextState
    return (flagSim, currentTime, currentState, timeState, transMatrix)

In [4]:
def OneHistory(para):
    '''Simulate one history of the system until the simulation time horizon is reached.
    '''
    # Intial conditions
    currentState = 3
    currentTime = 0
    flagSim = True
    # Counters initialization
    nState = 7
    timeState = np.zeros(nState)
    transMatrix = np.zeros((nState, nState))
    while flagSim:
        if (currentState==6):
            dT = RandExp(rate=para['r63'])
            nextState = 3
        elif (currentState==5):
            dT = RandExp(rate=para['r53'])
            nextState = 3        
        elif (currentState==4):
            dT = RandExp(rate=para['r43'])
            nextState = 3                    
        elif (currentState==3):
            dT = RandExp(rate=para['lambda32'])
            nextState = 2
        elif (currentState==2):
            dT26 = para['d26']
            dT21 = RandExp(rate=para['lambda21'])
            dT = np.min([dT26, dT21])
            if (dT==dT26):
                nextState = 6
            elif (dT==dT21):
                nextState = 1
            else:
                raise ValueError('Approximation error')
        elif (currentState==1):
            dT15 = para['d15']
            dT10 = RandExp(rate=para['lambda10'])
            dT = np.min([dT15, dT10])
            if (dT==dT15):
                nextState = 5
            elif (dT==dT10):
                nextState = 0
            else:
                raise ValueError('Approximation error')
        elif (currentState==0):
            dT = para['d04']
            nextState = 4
        else:
            raise ValueError('Unknown state')
        # Update variables
        (flagSim, currentTime, currentState, timeState, transMatrix) = UpdateVar(
            para['horizon'], currentTime, currentState, dT, nextState, timeState, transMatrix)
    return (timeState, transMatrix)            

In [5]:
def NHistories(nbN, para):
    '''Simulate several histories and aggregate time spent in the different states and achieved
    transitions (Monté-carlo approach).
    '''
    # Simulate a first history to assess output shapes
    (timeStateOne, transMatrixOne) = OneHistory(para)
    timeState = np.empty((nbN, timeStateOne.shape[0]))
    transMatrix = np.empty((nbN, transMatrixOne.shape[0], transMatrixOne.shape[1]))
    # Main loop
    for id in range(nbN):
        (timeState[id,:], transMatrix[id,:,:]) = OneHistory(para)
    return (timeState, transMatrix)

In [6]:
# Parmeters
nbHist = int(1e5)
para = {
    'horizon': 10*365*24,
    'lambda32': 5e-3,
    'lambda21': 2e-3,
    'lambda10': 1e-3,
    'd26': 168,  
    'd15': 336,
    'd04': 504,
    'r63': 1e-3,
    'r53': 1e-3,
    'r43': 1e-4}
# Simulation
(timeState, transMatrix) = NHistories(nbHist, para)
# Get simulations results

In [7]:
# Probability of each states
res = {}
for id in range(7):
    res['State{:d}'.format(id)] = np.mean(timeState[:, id]/np.sum(timeState[:, :], axis=1))
    print('State {:d} -> Probability: {:.5f}'.format(id, res['State{:d}'.format(id)], axis=1))
# Average simulated time
res['AveSimTime'] = np.mean(np.sum(timeState[:, :], axis=1))
print('Average simulated time (h): {:.2f}'.format(res['AveSimTime']))
# Average operating time
res['AveOpeTime'] = np.mean(np.sum(timeState[:, [1, 2, 3]], axis=1))
print('Average operating time (h): {:.2f}'.format(res['AveOpeTime']))
# Average repairing time
res['aveRepTime'] = np.mean(np.sum(timeState[:, [4, 5, 6]], axis=1))
print('Average repairing time (h): {:.2f}'.format(res['aveRepTime']))
# Average down time
res['aveDownTime'] = np.mean(np.sum(timeState[:, [0, 4, 5, 6]], axis=1))
print('Average down time (h): {:.2f}'.format(res['aveDownTime']))
# Average number of corrective maintenance
res['aveCorMaint'] = np.mean(transMatrix[:, 4, 3])
print('Average number of corrective maintenance: {:.2f}'.format(res['aveCorMaint']))
# Average number of preventive maintenance
res['avePreMaint'] = np.mean(transMatrix[:, 6, 3])+np.mean(transMatrix[:, 5, 3]) 
print('Average number of preventive maintenance: {:.2f}'.format(res['avePreMaint']))

State 0 -> Probability: 0.01951
State 1 -> Probability: 0.03884
State 2 -> Probability: 0.06828
State 3 -> Probability: 0.09570
State 4 -> Probability: 0.34357
State 5 -> Probability: 0.09595
State 6 -> Probability: 0.33815
Average simulated time (h): 87600.00
Average operating time (h): 17766.83
Average repairing time (h): 68124.50
Average down time (h): 69833.17
Average number of corrective maintenance: 3.38
Average number of preventive maintenance: 38.44
